### Creating CNN Model and Optimize it using Keras Tuner

In [1]:
!pip install keras-tuner

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

C:\Users\welcome\anaconda3\envs\deep_learning\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
print(tf.__version__)

2.9.1


In [4]:
fashion_mnist = keras.datasets.fashion_mnist

In [7]:
# training and test images
(train_images, train_labels),(test_images, test_labels) = fashion_mnist.load_data()

In [8]:
# scaling down to 0 to 1 to quickly train our model

train_images = train_images/255.0
test_images = test_images/255.0

In [10]:
train_images[0].shape

(28, 28)

In [11]:
# resize the pixel

train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [16]:
# creating a udf to build model

def build_model(hp):
    model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    
    return model

In [17]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [18]:
tuner_search = RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='output', project_name = 'Mnist_fashion')

In [19]:
tuner_search.search(train_images,train_labels, epochs = 3, validation_split = 0.1)

Trial 5 Complete [00h 08m 20s]
val_accuracy: 0.9116666913032532

Best val_accuracy So Far: 0.9116666913032532
Total elapsed time: 00h 26m 01s
INFO:tensorflow:Oracle triggered exit


In [20]:
model_bpm = tuner_search.get_best_models(num_models=1)[0]

In [21]:
model_bpm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 112)       1120      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 48)        134448    
                                                                 
 flatten (Flatten)           (None, 23232)             0         
                                                                 
 dense (Dense)               (None, 96)                2230368   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 2,366,906
Trainable params: 2,366,906
Non-trainable params: 0
_________________________________________________________________


In [22]:
model_bpm.fit(train_images,train_labels, epochs = 6, validation_split = 0.1, initial_epoch = 3)

Epoch 4/6
1688/1688 [==============================] - 171s 101ms/step - loss: 0.1345 - accuracy: 0.9498 - val_loss: 0.2667 - val_accuracy: 0.9138
Epoch 5/6
1688/1688 [==============================] - 166s 98ms/step - loss: 0.0945 - accuracy: 0.9639 - val_loss: 0.2780 - val_accuracy: 0.9205
Epoch 6/6
1688/1688 [==============================] - 165s 98ms/step - loss: 0.0670 - accuracy: 0.9761 - val_loss: 0.3462 - val_accuracy: 0.9168


In [ ]:
## for image augmentation

# from keras.preprocessing.image import ImageDataGenerator

# datagen = ImageDataGenerator(
#         rotation_range=40,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         rescale=1./255,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True,
#         fill_mode='nearest')